In [1]:
from qiskit.circuit.library.standard_gates import UGate
import random as rnd

# Returns a circuit representation of a random unitary matrix
def random_v_qubit_unitary(v):
    rand_u = QuantumCircuit(v)
    for i in range(v):
        theta = rnd.random() * 2 * 3.14
        phi = rnd.random() * 2 * 3.14
        lamb = rnd.random() * 2 * 3.14
        name = 'RU'
        random_gate = UGate(theta, phi, lamb)
        rand_u.append(random_gate, [i])
    return rand_u

In [2]:
# Returns decimal representation of bit string (fraction)
def bin_to_dec(str):
    value = 0
    for i in range(len(str)):
        value += float(str[i]) * 2**(-1-i)
    return value

In [3]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library import QFT
from qiskit.quantum_info import Operator

def phase_estimation_circuit(t, v, U):
    """ Circuit Initialisation"""
    # Create a new circuit for phase estimation with t + v qubits.
    # We will measure the precision qubits into a classical register.
    # We therefore need t classical bits (denoted by c in the diagram).
    phase_estimation_circuit = QuantumCircuit(t+v, t)

    """ Stage 1: Preparation of State for Inverse QFT"""
    # Apply Hadamard transform to first t qubits
    phase_estimation_circuit.h(range(t))

    # Apply Controlled-U gates for each qubit in t
    for i in range(t-1, -1, -1): # t-1 to 0
        for k in range(2**i): # 0 to 2^i
            """ Create an array of indices indicating which 'wires' to connect
            the unitary to in the circuit """
            qubits = [j for j in range(t-1, t+v)]
            qubits[0] = t-i-1
            # Create a single qubit controlled-U gate from the unitary.
            cru = U.control()
            # Apply the controlled-U gate to the desired wires
            phase_estimation_circuit.append(cru, qubits)

    """ Stage 2: Applincation of Inverse QFT"""  
    # Apply inverse quantum Fourier transform
    phase_estimation_circuit.barrier(range(t+v))
    inverse_qft = QFT(t, inverse=True)
    phase_estimation_circuit.append(inverse_qft.to_gate(), range(t))

    """ Stage 3: Measurement """
    # Measure each of t qubits
    phase_estimation_circuit.barrier(range(t+v))
    phase_estimation_circuit.measure(range(t), range(t))
    
    return phase_estimation_circuit

In [4]:
import numpy as np
from qiskit.providers.aer import QasmSimulator
from qiskit import transpile

""" Tunable Parameters"""
""" Guidelines: 
- v increases matrix dimension (dimension = 2^v)
- t increases accuracy
- Keep t > v, t >= v + 2 is recommended
- Simulation runtime is exponential in t and v. Simulations of t+v > 20 take minutes to run
"""
t = 3 # number of precision qubits
v = 2 # number of eigenstate qubits
simulation_count = 100 * 2**t # Number of times to repeat the experiment

# Create a random unitary implemente as a circuit
random_unitary = random_v_qubit_unitary(v)
random_unitary = QuantumCircuit(2)
random_unitary.x(0)
random_unitary.z(1)
# Build a circuit to perform quantum phase estimation on the given unitary
QPE_circuit = phase_estimation_circuit(t, v, random_unitary)

""" Simulation """
# Create a simulator
simulator = QasmSimulator()
# Compile the circuit for the simulator
compiled_circuit = transpile(QPE_circuit, simulator)
# Simulate the circuit
simulation = simulator.run(compiled_circuit, shots=simulation_count)
# Get the simulation results
simulation_result = simulation.result()
counts = simulation_result.get_counts(compiled_circuit)

""" Processing of Simulation Results """
# Sort results from highest to lowest
results_sorted = sorted(counts, key = lambda x: abs(counts[x])**2, reverse=True)
# Get most common results
if v <= t:
    most_common_results = results_sorted[:2**v]
else:
    most_common_results = results_sorted[:2**t]    
# Get eigenvalues in a + bi form.
eigenvalue_estimates = [np.exp(2j * np.pi * bin_to_dec(k)) for i, k in enumerate(most_common_results)]

""" Obtaining Eigenvalues Using Classical Algorithm """
# Get a numpy matrix representation of the unitary gate
U = np.array(Operator(random_unitary).data)
# Calculate the eigenvalues classically
true_eigenvalues, a = np.linalg.eig(U)

In [5]:
phases = sorted([np.arccos(true_eigenvalues[i].real) / (2 * np.pi) for i in range(len(true_eigenvalues))])
phases

[4.743186923619966e-09,
 4.743186923619966e-09,
 0.4999999976284065,
 0.4999999976284065]

In [6]:
phases_est = sorted([np.arccos(eigenvalue_estimates[i].real) / (2 * np.pi) for i in range(len(eigenvalue_estimates))])
phases_est

[0.0, 0.125, 0.12500000000000006, 0.37500000000000006]